In [1]:
import numpy as np
import pandas as pd

from datetime import date

In [2]:
def clean_dataframe(df):
    pattern = 'Fechamento\r\najust p/ prov\r\nEm moeda orig\r\n'
    df.columns = df.columns.str.replace(pattern, '')

    dates = pd.to_datetime(df['Data'], dayfirst=True)

    # Detach dates
    tickers = [col for col in df.columns if col != 'Data']
    df = df[tickers]

    # comma_to_dot = lambda x: x.replace('.', '').replace(',', '.')
    # df = df.applymap(comma_to_dot)
    df = df.replace('-', np.nan)
    df = df.apply(pd.to_numeric)

    # Reattach dates
    df['Data'] = dates
    
    # Reorder columns so that dates show up first
    cols = ['Data'] + list(df.columns.drop('Data'))
    df = df[cols]
    
    # Drop weekends and holidays
    df = df.dropna(axis=0, how='all', subset=tickers)
    
    return df

In [3]:
path = './../data/economatica.csv'
close = pd.read_csv(path, delimiter=',')

In [4]:
close = clean_dataframe(close)

In [5]:
close

,Data,RRRP3,TTEN3,QVUM3B,QVQP3B,APPA3,APPA4,ABCB3,ABCB4,ABCB11,...,WWOW3,ILMD3,ILMD4,YBRA3B,YDUQ3,ESTC4,ESTC11,OPZI3B,ZAMP3,ZIVI4
0,1986-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1986-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.916550e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1986-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.858220e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1986-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1986-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9610,2022-11-03,47.88,12.96,NaN,NaN,NaN,NaN,NaN,21.51,NaN,...,NaN,NaN,NaN,NaN,16.07,NaN,NaN,NaN,7.30,NaN
9611,2022-11-04,51.31,13.52,NaN,NaN,NaN,NaN,NaN,21.81,NaN,...,NaN,NaN,NaN,NaN,16.35,NaN,NaN,NaN,7.42,NaN
9612,2022-11-07,48.76,12.74,NaN,NaN,NaN,NaN,NaN,21.05,NaN,...,NaN,NaN,NaN,NaN,14.61,NaN,NaN,NaN,7.00,NaN
9613,2022-11-08,47.00,12.13,NaN,NaN,NaN,NaN,NaN,21.50,NaN,...,NaN,NaN,NaN,NaN,13.98,NaN,NaN,NaN,7.10,NaN


### Momentum

In [6]:
def list_returns(df, today, lookback=1):
    start = today - pd.DateOffset(months=lookback)
    _filter = df.set_index('Data').loc[start:today, :].dropna(axis='columns', how='any')
    return _filter.pct_change().add(1).cumprod().iloc[-1]

def get_momentum_assets(df, today, lookback=1, assets=10):
    returns = list_returns(df, today, lookback=lookback).sort_values(ascending=False)
    if len(returns) < assets:
        return returns.index
    return returns.index[:10]

In [7]:
get_momentum_assets(close, date(2022, 11, 1))

Index(['DEXP3', 'JSLG3', 'PRIO3', 'PRNR3', 'MILS3', 'RRRP3', 'ONCO3', 'CSED3',
       'CPLE3', 'CAMB3'],
      dtype='object')